# Мэтчинг товаров в онлайн-магазине

## Описание проекта

Заказчик - компания Samokat tech.

Цель исследования: находить соответствия между товарами из запроса и товарами из базы на основе векторных представлений товаров.

Требуется для каждого товара в запросе найти 10 ближайших товаров-соседей из базы.

Используемая метрика - recall.

## Описание данных

Заказчиком предоставлены датасеты:
1) База. 2.9 млн записей товаров в векторизированном виде. Каждая запись содержит ID товара и 72-мерный вектор - эмбеддинг.

2) Обучающий датасет. 100 тыс. записей - содержит эмбеддинги товаров, которым требуется найти соответствие в базе, и целевой признак - ID соответствующего товара из базы.

3) Тестовый датасет: содержит 100 тыс. эмбеддингов, отличных от трейна.

## Импорт библиотек

In [2]:
import faiss
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
from sklearn.utils import shuffle

## Загрузка и осмотр базы и обучающей выборки

In [3]:
%%time
base = pd.read_csv('base.csv')
# загружаем базу

Wall time: 1min 7s


In [19]:
base.info()
base.head()
# осматриваем базу

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2918139 entries, 0 to 2918138
Data columns (total 73 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Id      object 
 1   0       float64
 2   1       float64
 3   2       float64
 4   3       float64
 5   4       float64
 6   5       float64
 7   6       float64
 8   7       float64
 9   8       float64
 10  9       float64
 11  10      float64
 12  11      float64
 13  12      float64
 14  13      float64
 15  14      float64
 16  15      float64
 17  16      float64
 18  17      float64
 19  18      float64
 20  19      float64
 21  20      float64
 22  21      float64
 23  22      float64
 24  23      float64
 25  24      float64
 26  25      float64
 27  26      float64
 28  27      float64
 29  28      float64
 30  29      float64
 31  30      float64
 32  31      float64
 33  32      float64
 34  33      float64
 35  34      float64
 36  35      float64
 37  36      float64
 38  37      float64
 39  38      float64
 40  

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


In [4]:
base.isna().sum().sum()
# проверяем базу на наличие пропусков

0

In [6]:
base.duplicated().sum()
# проверяем базу на наличие дубликатов

0

In [7]:
train = pd.read_csv('train.csv')
# загружаем трэйн

In [20]:
train.info()
train.head()
# осматриваем трэйн

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 74 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      100000 non-null  float64
 21  20      100

,Id,0,1,2,3,4,5,6,7,8,...,63,64,65,66,67,68,69,70,71,Target
0,0-query,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,-347.360606,38.307602,109.085560,...,70.107360,-155.80257,-101.965943,65.903790,34.457500,62.642094,134.763600,-415.750254,-25.958572,675816-base
1,1-query,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-94.572749,-78.433090,124.915900,...,4.669178,-151.69771,-1.638704,68.170876,25.096191,89.974976,130.589630,-1035.092211,-51.276833,366656-base
2,2-query,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,26.943937,-30.447489,125.771164,...,78.039764,-169.14620,82.144186,66.008220,18.400496,212.409730,121.931470,-1074.464888,-22.547178,1447819-base
3,3-query,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,-775.150134,79.186520,124.003100,...,44.515266,-145.41675,93.990981,64.131350,106.061920,83.178760,118.277725,-1074.464888,-19.902788,1472602-base
4,4-query,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,-321.317964,45.984676,125.941284,...,45.028910,-196.09207,-117.626337,66.926220,42.456170,77.621765,92.479930,-1074.464888,-21.149351,717819-base


In [8]:
train.isna().sum().sum()

0

In [9]:
train.duplicated().sum()

0

In [10]:
test = pd.read_csv('test.csv')
# загружаем тест

In [21]:
test.info()
test.head()
# осматриваем тест

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 73 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Id      100000 non-null  object 
 1   0       100000 non-null  float64
 2   1       100000 non-null  float64
 3   2       100000 non-null  float64
 4   3       100000 non-null  float64
 5   4       100000 non-null  float64
 6   5       100000 non-null  float64
 7   6       100000 non-null  float64
 8   7       100000 non-null  float64
 9   8       100000 non-null  float64
 10  9       100000 non-null  float64
 11  10      100000 non-null  float64
 12  11      100000 non-null  float64
 13  12      100000 non-null  float64
 14  13      100000 non-null  float64
 15  14      100000 non-null  float64
 16  15      100000 non-null  float64
 17  16      100000 non-null  float64
 18  17      100000 non-null  float64
 19  18      100000 non-null  float64
 20  19      100000 non-null  float64
 21  20      100

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,100000-query,-57.372734,3.597752,-13.213642,-125.92679,110.74594,-81.279594,-461.003172,139.815720,112.880980,...,-75.51302,52.830902,-143.43945,59.051935,69.28224,61.927513,111.59253,115.140656,-1099.130485,-117.079360
1,100001-query,-53.758705,12.790300,-43.268543,-134.41762,114.44991,-90.520130,-759.626065,63.995087,127.117905,...,-79.44183,29.185436,-168.60590,-82.872443,70.76560,-65.975950,97.07716,123.391640,-744.442332,-25.009320
2,100002-query,-64.175095,-3.980927,-7.679249,-170.16093,96.44616,-62.377740,-759.626065,87.477554,131.270110,...,-134.79541,37.368730,-159.66231,-119.232725,67.71044,86.002060,137.63641,141.081630,-294.052271,-70.969604
3,100003-query,-99.286860,16.123936,9.837166,-148.06044,83.69708,-133.729720,58.576403,-19.046660,115.042404,...,-77.23611,44.100494,-132.53012,-106.318982,70.88396,23.577892,133.18396,143.252940,-799.363667,-89.392670
4,100004-query,-79.532920,-0.364173,-16.027431,-170.88495,165.45392,-28.291668,33.931936,34.411217,128.903980,...,-123.77025,45.635944,-134.25893,13.735359,70.61763,15.332115,154.56812,101.700640,-1171.892332,-125.307890


In [11]:
test.isna().sum().sum()


0

In [12]:
test.duplicated().sum()

0

Ниже проверим, все ли значения столбца train['Target'] присутствуют в базе:

In [13]:
base.loc[base['Id'].isin(train['Target'])]['Id'].count()
# количество записей в ID из базы, которые присутствуют в целевом признаке из трэйна

91794

In [14]:
train.loc[train['Target'].isin(base['Id'])]['Target'].count()
# количество записей в целевом признаке трэйна, для которых есть соответствие в базе

100000

Как видно, 8 с небольшим тысяч записей трейна не имеют уникального таргета в базе. Следовательно, таргет имеет свойство повторяться. 

In [10]:
train['Target'].value_counts().head(20)
# смотрим, насколько часто повторяется целевой признак

41568-base     7
18784-base     7
106681-base    6
7473-base      6
90018-base     6
328932-base    5
113723-base    5
132878-base    5
46246-base     5
97372-base     5
245081-base    5
141489-base    5
121075-base    5
37032-base     5
324246-base    5
173185-base    5
297371-base    5
282745-base    5
412645-base    5
275969-base    5
Name: Target, dtype: int64

In [11]:
train['Target'].value_counts().sum()


100000

Проверено: некоторые значения в поле Target не уникальны, но это не помешает дальнейшей работе - все они присутствуют в базе.

## Алгоритм решения

Для решения задачи мэтчинга предложен следующий алгоритм:
1) По id из поля train['Target'] получаем векторы из базы.

2) Все 72 измерения трейна используем как обучающие признаки.

3) Каждая размерность вектора из базы становится отдельным целевым признаком.

4) Обучаем 72 модели: фичи у всех одинаковые, таргет у каждой свой - отдельная размерность векторов из базы.

5) Из них выбираем предсказания тех размерностей, где f2-значение показывает удовлетворительный результат (какой именно - увидим в процессе). Прочие размерности удаляем.

6) Из полученных столбцов - предсказаний размерностей составляем строки - векторы. 

7) Из базы также удаляем неиспользуемые размерности

8) К вектору предсказаний находим 10 ближайших соседей в базе

9) Результат сохраняем в CSV с соблюдением требований формы для проверки заказчиком

## Решение

### Формируем признаки для обучающей выборки
#### Целевые признаки

In [17]:
target_train = pd.DataFrame(train['Target'])
# получаем ID целевых записей в базе

base = base.set_index('Id')
# для объединения таблиц индекс базы временно подменим на ID

target_train = target_train.join(base, on='Target', how='left')
# объединяем таблицы

base = base.reset_index()
# возвращаем индекс на место - он еще понадобится.

target_train = target_train.drop('Target', axis=1)
# убираем ненужный столбец - с сохранением порядка строк

target_train.columns
# проверяем полученные признаки

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71'],
      dtype='object')

Полученные признаки соответствуют ожидаемым.

#### Обучающие признаки 

In [22]:
features_train = train.drop(['Id','Target'], axis=1)
features_train.columns
# проверяем полученные признаки

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60',
       '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71'],
      dtype='object')

Обучающие признаки также соответствуют ожидаемым

#### Обработка тестовой выборки

In [43]:
features_test = test.drop('Id', axis=1)
# удаляем ID
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   6       100000 non-null  float64
 7   7       100000 non-null  float64
 8   8       100000 non-null  float64
 9   9       100000 non-null  float64
 10  10      100000 non-null  float64
 11  11      100000 non-null  float64
 12  12      100000 non-null  float64
 13  13      100000 non-null  float64
 14  14      100000 non-null  float64
 15  15      100000 non-null  float64
 16  16      100000 non-null  float64
 17  17      100000 non-null  float64
 18  18      100000 non-null  float64
 19  19      100000 non-null  float64
 20  20      100000 non-null  float64
 21  21      100

### Обучение модели 

#### Определим приблизительный диапазон гиперпараметров на одной размерности



In [23]:
# param_grid = {'max_depth':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,15, 20, 25, -1],
#              'n_estimators': [10,50, 100, 150, 200, 250, 300],
#              'num_leaves':[6, 10, 20,31, 41, 51],
#              'random_state':[13]}
# при первичном подборе происходил перебор всех этих параметров.
# ниже - сразу введены лучшие параметры для быстродействия

param_grid = {'max_depth':[8],
             'n_estimators': [250],
             'num_leaves':[51],
             'random_state':[13]}

In [24]:
# Задаем GridSearch с моделью LGBMRegressor:

model = GridSearchCV(estimator=LGBMRegressor(),
                    param_grid=param_grid,
                    scoring='r2',
                    cv=3,
                    n_jobs=-1,
                    verbose=3)

In [25]:
%%time
model.fit(features_train, target_train.loc[:,'0']) # обучаем на фичах и первом столбце таргета
print(model.best_score_) # выводим лучший результат
print(model.best_params_) # и лучшие гиперпараметры

Fitting 3 folds for each of 1 candidates, totalling 3 fits
0.7302624916187471
{'max_depth': 8, 'n_estimators': 250, 'num_leaves': 51, 'random_state': 13}
Wall time: 43.8 s


Следующие параметры оказались оптимальными:

{'max_depth': 8, 'n_estimators': 250, 'num_leaves': 51, 'random_state': 13}

При этом метрика r2 составила  0.73.

При подборе параметров регрессора для каждой размерности будем использовать близкие значения - для экономии времени. 

#### Обучаем по модели на каджой размерности, получаем предсказания и сохраняем в таблице

In [28]:
# пустая таблица
predictions = pd.DataFrame()

In [29]:
# грид из близких параметров
new_grid = {'max_depth':[7, 8, 9],
             'n_estimators': [230, 250, 270],
             'num_leaves':[46, 51, 57],
             'random_state':[13]}

In [30]:
%%time
for n in target_train.columns: # для каждой размерности целевых признаков
    model = GridSearchCV(estimator=LGBMRegressor(), # создаём и модель с подбором параметров и кросс-валидацией
                    param_grid=new_grid,
                    scoring='r2',
                    cv=3,
                    n_jobs=-1,
                    verbose=3)
    model.fit(features_train, target_train.loc[:, n]) # обучаем данную модель
    predictions[n] = model.predict(features_test) # предсказываем размерность 'n' и записываем ее в таблицу predictions
    print(n) # выводим номер размерности
    print(model.best_score_) # выводим лучшее значение метрики
    print(model.best_params_) # отображаем лучшие параметры

Fitting 3 folds for each of 27 candidates, totalling 81 fits
0
0.7308824514597659
{'max_depth': 8, 'n_estimators': 230, 'num_leaves': 46, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
1
0.7350989189043906
{'max_depth': 8, 'n_estimators': 270, 'num_leaves': 51, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
2
0.7193666971905408
{'max_depth': 8, 'n_estimators': 230, 'num_leaves': 51, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
3
0.7368909796386186
{'max_depth': 8, 'n_estimators': 230, 'num_leaves': 57, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
4
0.7261054911733235
{'max_depth': 9, 'n_estimators': 230, 'num_leaves': 46, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
5
0.7324768598385832
{'max_depth': 9, 'n_estimators': 250, 'num_leaves': 57, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling

52
0.7295631188981536
{'max_depth': 8, 'n_estimators': 270, 'num_leaves': 51, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
53
0.7320415274681255
{'max_depth': 7, 'n_estimators': 230, 'num_leaves': 46, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
54
0.7167533266431588
{'max_depth': 7, 'n_estimators': 250, 'num_leaves': 51, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
55
0.7365877217694266
{'max_depth': 7, 'n_estimators': 230, 'num_leaves': 46, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
56
0.7444893080911279
{'max_depth': 9, 'n_estimators': 230, 'num_leaves': 51, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
57
0.7242333373247097
{'max_depth': 8, 'n_estimators': 270, 'num_leaves': 57, 'random_state': 13}
Fitting 3 folds for each of 27 candidates, totalling 81 fits
58
0.7338932655910156
{'max_depth': 7, 'n_esti

Столбцы с низким качеством предсказаний:
6, 21, 25, 33, 44, 59, 65, 70.

#### Сохраняем полученные предсказания в csv

In [34]:
# учитывая что предсказание заняло 8 часов, сохраним данные в csv, чтоб в дальнейшем не приходилось повторять запуск
predictions.to_csv('lgbm_predictions.csv', index=False)

In [35]:
predictions = pd.read_csv('lgbm_predictions.csv')

#### Удаляем размерности с низким качеством предсказания:

In [27]:
# судя по выводу кода из ячейки с обучением моделей, качество предсказаний в данных столбцах значимо хуже остальных 
columns_to_drop = ['6', '21', '25', '33', '44', '59', '65', '70']

In [37]:
predictions_cropped = predictions.drop(columns_to_drop, axis=1)
# удаляем вышеуказанные столбцы


In [39]:
predictions_cropped.info()
# оцениваем результат
predictions_cropped.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 64 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       100000 non-null  float64
 1   1       100000 non-null  float64
 2   2       100000 non-null  float64
 3   3       100000 non-null  float64
 4   4       100000 non-null  float64
 5   5       100000 non-null  float64
 6   7       100000 non-null  float64
 7   8       100000 non-null  float64
 8   9       100000 non-null  float64
 9   10      100000 non-null  float64
 10  11      100000 non-null  float64
 11  12      100000 non-null  float64
 12  13      100000 non-null  float64
 13  14      100000 non-null  float64
 14  15      100000 non-null  float64
 15  16      100000 non-null  float64
 16  17      100000 non-null  float64
 17  18      100000 non-null  float64
 18  19      100000 non-null  float64
 19  20      100000 non-null  float64
 20  22      100000 non-null  float64
 21  23      100

(100000, 64)

Содержание таблицы predictions_cropped соответствует ожидаемому.

### Поиск ближайших соседей



В рамках данной работы было опробовано несколько методов поиска ближайших соседей. Лучший результат показал способ с полным перебором:

In [27]:
base_vectors = base.drop(['Id'] + columns_to_drop, axis=1).values
# получаем нужные столбцы из базы

In [30]:
base_vectors.shape
# проверяем размерность полученной выгрузки

(2918139, 64)

In [31]:
index = faiss.IndexFlatL2(64)
# создаем объект faiss.IndexFlatL2 размерностью 64
index.add(base_vectors)
# добавляем данные из базы

In [39]:
%%time
D, I = index.search(predictions_cropped, 10)
# ищем 10 ближайших соответствий каждой строке из predictions_cropped

Wall time: 18min


### Составление файла с результатом

In [40]:
def prediction(row):
    return(" ".join(base.loc[row,:]['Id']))
# функция на вход будет получать строку из таблицы,
# затем значения из строки будет искать в индексах таблицы,
# возвращать строку, составленную из ID в базе через пробел (согласно образцу ответа, предоставленному заказчиком) 

In [41]:
result = pd.DataFrame(I)
# для упрощения работы с данными переводим array в таблицу pandas

In [42]:
result['Predicted'] = result.apply(prediction, axis=1) # применяем функцию

In [43]:
result['Predicted'] 
# проверяем результат работы функции

0        2341758-base 399677-base 2760762-base 368296-b...
1        2666508-base 163485-base 11853-base 508295-bas...
2        472256-base 496010-base 153272-base 25113-base...
3        3168654-base 2177262-base 1831175-base 4473809...
4        1217188-base 75484-base 2411488-base 2980294-b...
                               ...                        
99995    1533158-base 880713-base 2681222-base 1243195-...
99996    925466-base 936053-base 272706-base 525997-bas...
99997    1801591-base 1196513-base 1517783-base 646953-...
99998    341779-base 305868-base 4523822-base 3577160-b...
99999    4678196-base 1011592-base 2503531-base 2818124...
Name: Predicted, Length: 100000, dtype: object

In [44]:
result['Id'] = test['Id']
# добавляем столбец с Id из тестовой выборки

In [45]:
result = result[['Id', 'Predicted']]
# оставляем только нужные столбцы

In [46]:
result

,Id,Predicted
0,100000-query,2341758-base 399677-base 2760762-base 368296-b...
1,100001-query,2666508-base 163485-base 11853-base 508295-bas...
2,100002-query,472256-base 496010-base 153272-base 25113-base...
3,100003-query,3168654-base 2177262-base 1831175-base 4473809...
4,100004-query,1217188-base 75484-base 2411488-base 2980294-b...
...,...,...
99995,199995-query,1533158-base 880713-base 2681222-base 1243195-...
99996,199996-query,925466-base 936053-base 272706-base 525997-bas...
99997,199997-query,1801591-base 1196513-base 1517783-base 646953-...
99998,199998-query,341779-base 305868-base 4523822-base 3577160-b...


In [47]:
result.to_csv('result_cropped_full.csv', index=False)
# сохраняем результат в csv

### Результат работы алгоритма 
Предложенным выше методом достигнута полнота 0.73 на тестовых данных при загрузке результата на kaggle.

## Метод повышения качества алгоритма

С целью улучшения качества предсказаний предложено добавить в алгоритм модель третьего уровня: 

- Обучаем классификатор
    - положительным классом будут соответствия между вектором трэйн-выборки и из базы;
    - отрицательным классом будут подобранные случайным образом соответствия
- Делаем иную выгрузку из базы
    - для каждой записи из query ищем не 10, а более 10 ближайших соседей из базы
    - используем классификатор двумя способами:
        - посредством метода классификатора predict_proba выбираем 10 записей с наибольшей вероятностью положительного класса. 
        - используем метод классификатора predict c целью снижения приоритета отрицательного класса.
        - сравниваем результаты между собой и с исходным результатом

### Подготовим данные

In [103]:
features_class = features_train[:50000].drop(columns_to_drop, axis=1).join(
    target_train[:50000].drop(columns_to_drop, axis=1), rsuffix='t')
# первая половина обучающей выборки для классификатора:
# соединяем половину трэйн-выборки с соответствующими таргету записями из таргета.

In [104]:
features_class['target'] = 1
# присваиваем положительный класс

In [105]:
features_class.shape
# проверяем число столбцов: должно быть 129 - в строке два 64-размерных вектора и один столбец класса

(50000, 129)

In [106]:
features_class
# осматриваем результат

,0,1,2,3,4,5,7,8,9,10,...,61t,62t,63t,64t,66t,67t,68t,69t,71t,target
0,-53.882748,17.971436,-42.117104,-183.93668,187.517490,-87.144930,38.307602,109.085560,30.413513,-88.082690,...,-136.331360,-107.259094,80.318184,-151.78957,66.886116,45.391710,40.096794,116.982010,-18.195980,1
1,-87.776370,6.806268,-32.054546,-177.26039,120.803330,-83.810590,-78.433090,124.915900,140.331070,-177.605800,...,-132.142880,-77.159775,12.424929,-162.74518,70.269700,-15.091993,48.642250,101.123980,-71.665270,1
2,-49.979565,3.841486,-116.118590,-180.40198,190.128430,-50.837620,-30.447489,125.771164,211.607820,-86.346560,...,-145.316040,-51.152130,94.640840,-176.32103,66.266970,17.248710,170.587340,131.261600,-33.020058,1
3,-47.810562,9.086598,-115.401695,-121.01136,94.652840,-109.255410,79.186520,124.003100,242.650650,-146.517070,...,-127.109840,-56.015300,41.230500,-136.19551,64.016380,97.340790,54.679436,118.847440,3.276848,1
4,-79.632126,14.442886,-58.903397,-147.05254,57.127068,-16.239529,45.984676,125.941284,103.392670,-107.153020,...,-103.342026,-68.775360,45.028675,-196.09872,66.926575,42.453735,77.619740,92.482574,-21.129560,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-57.257410,2.890785,-50.811123,-118.41748,8.520401,-81.084350,55.407425,117.723660,230.292720,-142.937560,...,-129.294460,-23.869743,71.260330,-180.09415,67.617676,39.593010,52.107370,71.998870,-58.449375,1
49996,-54.996090,9.204070,-31.005787,-159.88655,201.896410,-89.823990,-59.827393,126.782845,133.640640,-169.872480,...,-140.378100,-114.298010,3.899488,-166.42350,66.568220,7.049480,62.138836,132.433150,-73.210810,1
49997,-87.809620,16.104704,-9.053341,-133.52360,204.127140,-81.612686,-43.382545,123.780525,36.190895,-189.743990,...,-124.267540,-83.360230,10.408098,-209.89703,69.190636,-76.470140,60.357050,118.961680,-38.207077,1
49998,-88.828350,6.122507,-57.968323,-143.61010,72.058205,-81.916470,3.348228,126.298600,96.652756,-111.561295,...,-140.731340,-99.281110,23.797993,-153.91867,65.864320,40.747856,181.227510,133.742450,-92.536960,1


In [108]:
neg_class = pd.concat([features_train[50000:].drop(columns_to_drop, axis=1).reset_index(drop=True),
           base.drop('Id', axis=1).sample(n=50000, random_state=13).drop(columns_to_drop, axis=1).reset_index(drop=True)],
          ignore_index=True, axis=1)

# получаем негативный класс:
# cоединяем вторую половину features_train с 50000 случайными записями из базы.

In [109]:
neg_class['target'] = 0
neg_class.columns = features_class.columns
# присваиваем отрицательный класс
# присваиваем столбцам наименования, аналогичные первой половине выборки

In [110]:
neg_class.shape
# проверяем размер

(50000, 129)

In [111]:
features_class = shuffle(pd.concat([features_class, neg_class]), random_state=13)
# соединяем таблицы положительного и отрицательного классов, перемешиваем

In [112]:
features_class.shape

(100000, 129)

In [113]:
target_class = features_class['target']
# выделение целевого признака

In [114]:
features_class = features_class.drop('target', axis=1)
# выделение обучающих признаков

### Обучим классификатор

In [44]:
grid = {'max_depth':[5],
       'n_estimators': [10, 30, 50, 70, 110, 160, 220, 300, 400, 500],
        'num_leaves':[21, 31, 41],
        'random_state':[13]}
# задаем решетку параметров

In [126]:
clf = GridSearchCV(estimator=LGBMClassifier(),
                  param_grid=grid,
                    scoring='recall',
                  cv=3,
                  verbose=3,
                  n_jobs=-1)
# классификатор с поиском параметров и кросс-валидацией.
# scoring соответствует тому, что используется заказчиком исследования.

In [127]:
%%time
clf.fit(features_class, target_class)
# обучаем модель

Fitting 3 folds for each of 180 candidates, totalling 540 fits
Wall time: 53min 32s


GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'max_depth': [2, 5, 7, 10, 15, 20],
                         'n_estimators': [10, 30, 50, 70, 110, 160, 220, 300,
                                          400, 500],
                         'num_leaves': [21, 31, 41], 'random_state': [13]},
             scoring='recall', verbose=3)

In [128]:
clf.best_score_
# лучшее значение метрики

0.9274600227424229

In [129]:
clf.best_params_
# гиперпараметры при лучшем значении метрики. 

{'max_depth': 5, 'n_estimators': 500, 'num_leaves': 21, 'random_state': 13}

Классификатор показал лучший результат полноты 0.93 при гиперпараметрах: макс. глубина 5, количество эстиматоров 500, количество листьев 21.

### Ищем 20 ближайших соседей

In [453]:
%%time
D, I = index.search(predictions_cropped, 20)

Wall time: 18min 47s


In [454]:
pd.DataFrame(I).to_csv('20_neighbours.csv')
# сохраняем результат чтоб при повторном запуске кода не пришлось ждать

### Ранжируем соседей с помощью классификатора

In [455]:
neighbors = pd.DataFrame(I)
# для упрощения работы с данными сохраним полученный массив в dataframe

In [456]:
neighbors.shape

(100000, 20)

In [45]:
features_test = features_test.drop(columns_to_drop,axis=1)
# features_test для этого алгоритма не должен содержать столбцов, которых нет в базе

In [442]:
%%time
answer_proba = np.empty(0)  # пустой массив
for i in neighbors.index: # для каждой записи в neighbors выполняется следующее:
    probs = pd.DataFrame(data=np.array(neighbors.loc[i,:]), columns=['index_in_base'])
    # cоздаем таблицу, где в одном столбце будет индекс строки в базе,
    # в другом столбце классификатором будет рассчитана вероятность положительного класса для этой записи
    for j in probs.index:
        probs.loc[j, 'probability'] = clf.predict_proba(np.append(features_test.loc[i,:].values,
                                                              base_vectors[int(probs.loc[j,'index_in_base'])]).reshape(1, -1))[0][1]
    probs = probs.sort_values(by='probability', ascending=False).head(10)
    # выше выполнено следующее: соединена пара вектора из тестовой выборки и соответствующего вектора из базы. для нее
    # подсчитана вероятность положительного класса, то есть совпадения.
    # затем записи ранжированы по вероятности положительного класса, оставлен топ-10.
    answer_proba = np.append(answer_proba,(" ".join(base.loc[probs['index_in_base'],'Id'])))
    # далее по индексам в базе выгружены Id и связаны в строку, как в основном решении.
    # полученная строка добавлена к массиву.
    

Wall time: 1h 5min 32s


In [46]:
answer_class = np.empty(0)
for i in neighbors.index:
    probs = pd.DataFrame(data=np.array(neighbors.loc[i,:]), columns=['index_in_base'])
    for j in probs.index:
        probs.loc[j, 'probability'] = clf.predict(np.append(features_test.loc[i,:].values,
                                                              base_vectors[int(probs.loc[j,'index_in_base'])]).reshape(1, -1))
    probs = probs.sort_values(by='probability', ascending=False).head(10)
    answer_class = np.append(answer_class,(" ".join(base.loc[probs['index_in_base'],'Id'])))

    # здесь повторяется предыдущий алгоритм с одним отличием:
    # вместо ранжирования по вероятности происходит ранжирование по классу:
    # таким образом "отодвинуты" будут только записи с отрицательным классом,
    # прочие же останутся на своих местах.

In [445]:
result_prob = pd.DataFrame(index=test['Id'], data=answer_proba) 
# получаем ответ из массива с ранжированием по вероятности положительного класса

In [446]:
result_class = pd.DataFrame(index=test['Id'], data=answer_class)
# получаем ответ из массива с ранжированием по классу

In [447]:
result_prob = result_prob.reset_index()
result_class = result_class.reset_index()

In [448]:
result_prob.columns = ['Id', 'Predicted']
result_class.columns = ['Id', 'Predicted']

### Экспортируем результат в CSV

In [449]:
result_prob.to_csv('result_fin_prob.csv', index=False)
result_class.to_csv('result_fin_class_20.csv', index=False)

In [451]:
result_prob

,Id,Predicted
0,100000-query,1542803-base 368296-base 3209652-base 3839597-...
1,100001-query,2666508-base 508295-base 11853-base 163485-bas...
2,100002-query,472256-base 496010-base 25113-base 153272-base...
3,100003-query,3168654-base 1831175-base 2345993-base 1274091...
4,100004-query,1217188-base 75484-base 2863148-base 211178-ba...
...,...,...
99995,199995-query,2681222-base 1533158-base 2757076-base 880713-...
99996,199996-query,2653840-base 124195-base 272706-base 220273-ba...
99997,199997-query,1801591-base 646953-base 2931054-base 2544583-...
99998,199998-query,341779-base 4523822-base 3024721-base 4499729-...


In [452]:
result_class

,Id,Predicted
0,100000-query,2341758-base 399677-base 2760762-base 368296-b...
1,100001-query,2666508-base 163485-base 11853-base 508295-bas...
2,100002-query,472256-base 496010-base 153272-base 25113-base...
3,100003-query,3168654-base 2177262-base 1831175-base 4473809...
4,100004-query,1217188-base 75484-base 2411488-base 2980294-b...
...,...,...
99995,199995-query,1533158-base 880713-base 2681222-base 1243195-...
99996,199996-query,925466-base 936053-base 272706-base 525997-bas...
99997,199997-query,1801591-base 1517783-base 646953-base 2544583-...
99998,199998-query,341779-base 305868-base 4523822-base 1514452-b...


#### Результат использования усовершенствованного алгоритма

- При ранжировании по вероятности положительного класса полнота снизилась на 2%
- При ранжировании по классу полнота увеличилась на десятые доли процента, осталась приблизительно равной 73%.

Вывод: алгоритм с моделью третьего уровня в настоящее время не рекомендован к использованию.

## Вывод

### Цель работы: 
Предложить метод улучшения мэтчинга в онлайн-магазине.

Для каждого вектора в запросе найти 10 ближайших соответствий из базы.

Используемая метрика - recall.

### Материалы и методы:

Работа выполнена средствами Python, библиотек Pandas, NumPy, Faiss.
Задачи машинного обучения решались с помощью библиотек Sklearn, LightGBM.

### Ход выполнения проекта:

- Загружены и подготовлены данные.
- Предложен и выполнен двухуровневый алгоритм: предсказание векторов-соответствий товаров средствами, затем поиск 10 ближайших соответствий из базы к каждому предсказанию.
- С целью совершенствования алгоритма добавлена модель третьего уровня: классификатор, ранжирующий результаты по уровню соответствия.

### Результат работы:

- Двухуровневый алгоритм показал полноту 0.73 на тестовой выборке.
- С помощью модели третьего уровня достичь улучшения результата не удалось.

